<a href="https://colab.research.google.com/github/keenranger/various_things/blob/master/%EC%8A%B9%EC%9E%AC%EA%B3%BC%EC%A0%9C_mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from skimage import transform

tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
def resize_batch(imgs):
    imgs = imgs.reshape((-1, 28, 28, 1))
    resized_imgs = np.zeros((imgs.shape[0], 32, 32, 1))
    for i in range(imgs.shape[0]):
        resized_imgs[i, ..., 0] = transform.resize(imgs[i, ..., 0], (32, 32))
    return resized_imgs

In [19]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 1])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])


In [0]:
W1 = tf.Variable(tf.random_normal([5,5,1,6], stddev=0.01,dtype=tf.float32), tf.contrib.layers.xavier_initializer_conv2d())
conv2d_1=tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='VALID')
relu_1=tf.nn.relu(conv2d_1)
pool_1= tf.nn.max_pool(relu_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([5,5,6,16], stddev=0.01,dtype=tf.float32), tf.contrib.layers.xavier_initializer_conv2d())
conv2d_2 = tf.nn.conv2d(pool_1, W2, strides=[1, 1, 1, 1], padding='VALID')
relu_2=tf.nn.relu(conv2d_2)
pool_2 = tf.nn.max_pool(relu_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

FC=tf.contrib.layers.flatten(pool_2)

In [0]:
W3=tf.Variable(tf.random_normal([400, 120],stddev=0.01,dtype=tf.float32), tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.zeros([120],dtype=tf.float32))
Z1= tf.nn.relu(tf.matmul(FC,W3)+b3)

W4=tf.Variable(tf.random_normal([120, 84],stddev=0.01,dtype=tf.float32), tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.zeros([84],dtype=tf.float32))
Z2= tf.nn.relu(tf.matmul(Z1,W4)+b4)

W5=tf.Variable(tf.random_normal([84, 10],stddev=0.01,dtype=tf.float32), tf.contrib.layers.xavier_initializer())
b5=tf.Variable(tf.zeros([10],dtype=tf.float32))
S= tf.matmul(Z2,W5)+b5

In [0]:
hypothesis = tf.nn.softmax_cross_entropy_with_logits_v2(logits=S, labels=Y)

loss = tf.reduce_mean(hypothesis)

corrects = tf.equal(tf.argmax(S, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))


In [26]:
LEARNING_RATE = 0.0005
EPOCHS = 20
BATCH_SIZE = 100

optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(EPOCHS):
        for temp in range(int(mnist.train.num_examples / BATCH_SIZE)):
            X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
           # print(X_batch.shape)
            X_batch = resize_batch(X_batch)
            #print(X_batch.shape)
            sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
        if (epoch+1) % 1 == 0:
            acc_val = sess.run(accuracy,
              feed_dict={X: resize_batch(mnist.test.images),
                  Y: mnist.test.labels})
        
            print("epoch: {:04d}, accuracy: {:.4f}".format(epoch+1, acc_val))
    print("Learning finished.")

     
                                    

epoch: 0001, accuracy: 0.8893
epoch: 0002, accuracy: 0.9395
epoch: 0003, accuracy: 0.9580
epoch: 0004, accuracy: 0.9614
epoch: 0005, accuracy: 0.9715
epoch: 0006, accuracy: 0.9746
epoch: 0007, accuracy: 0.9769
epoch: 0008, accuracy: 0.9803
epoch: 0009, accuracy: 0.9814
epoch: 0010, accuracy: 0.9844
epoch: 0011, accuracy: 0.9859
epoch: 0012, accuracy: 0.9835
epoch: 0013, accuracy: 0.9872
epoch: 0014, accuracy: 0.9864
epoch: 0015, accuracy: 0.9863
epoch: 0016, accuracy: 0.9891
epoch: 0017, accuracy: 0.9861
epoch: 0018, accuracy: 0.9885
epoch: 0019, accuracy: 0.9863
epoch: 0020, accuracy: 0.9868
Learning finished.
